In [64]:
counter = 1

Run everything below once to generate a recipe, or multiple times to generate multiple recipes. If you want to generate multiple, the cell above needs to be ran just once, to initialise a counter and adjust the naming scheme for the recipe, which will be outputted into a file.

In [ ]:
import json
import numpy as np
from geneticalgorithm import geneticalgorithm as ga
from fractions import Fraction

# Load the knowledge base
with open('base.json', 'r') as file:
    knowledge_base = json.load(file)

# Extract ingredients and steps
ingredients = []
steps = []
for recipe in knowledge_base:
    for ingredient in recipe['ingredients']:
        if ingredient['name'] not in [ing['name'] for ing in ingredients]:
            ingredients.append(ingredient)
    for step in recipe['steps']:
        if step not in steps:
            steps.append(step)

num_ingredients = len(ingredients)
num_steps = len(steps)

# Critical steps that must appear
critical_steps = ["Preheat oven", "Mix dry ingredients", "Mix wet ingredients", "Combine wet and dry ingredients", "Bake"]

# Define the bounds for the genetic algorithm
varbound = np.array([[0, 1]] * num_ingredients + [[0, num_steps - 1]] * num_steps)

# Conversion factors to grams
conversion_factors = {
    "flour": 120,  # grams per cup
    "sugar": 200,  # grams per cup
    "almond milk": 240,  # grams per cup
    "oil": 220,  # grams per cup
    "cocoa powder": 100,  # grams per cup
    "all-purpose flour": 120,  # grams per cup
    "brown sugar": 200,  # grams per cup
    "cocoa powder": 100,  # grams per cup
    "plant-based milk": 240,  # grams per cup
    "vegetable oil": 220,  # grams per cup
    "maple syrup": 320,  # grams per cup
    "vanilla extract": 208,  # grams per cup
    "baking powder": 4.8,  # grams per teaspoon
    "salt": 6,  # grams per teaspoon
    "vegan chocolate chips": 170,  # grams per cup
    "almond flour": 96,  # grams per cup
    "sugar": 200,  # grams per cup
    "coconut sugar": 192,  # grams per cup
    "melted coconut oil": 220,  # grams per cup
    "almond milk": 240,  # grams per cup
    "baking soda": 4.8,  # grams per teaspoon
    "sourdough starter": 240,  # grams per cup
    "almond butter": 256,  # grams per cup
    "flour": 120,  # grams per cup
    "aquafaba": 240,  # grams per cup
    "cane sugar": 200,  # grams per cup
    "dark chocolate": 170,  # grams per cup
    "vegan butter": 227,  # grams per cup
    "Dutch cocoa powder": 100,  # grams per cup
    "sea salt": 6,  # grams per teaspoon
    "espresso powder": 2.4,  # grams per teaspoon
    "chocolate chips": 170  # grams per cup
}

# Function to convert ingredient amounts to grams
def convert_to_grams(amount, ingredient):
    try:
        value = float(Fraction(amount.split()[0]))
        unit = amount.split()[1]
        if unit == "cup" or unit == "cups":
            return value * conversion_factors.get(ingredient, 1)
        
        return value
    except (ValueError, IndexError):
        return 0.0

# Decode the recipe
def decode_recipe(X):
    recipe_ingredients = {ingredients[i]['name']: convert_to_grams(ingredients[i]['amount'], ingredients[i]['name']) for i in range(num_ingredients) if X[i] > 0}
    recipe_steps = [steps[int(X[i])] for i in range(num_ingredients, len(X)) if 0 <= int(X[i]) < num_steps]
    return recipe_ingredients, recipe_steps

# Normalize steps to reduce redundancy and include time and temperature
def normalize_steps(steps):
    normalization_map = {
        "Mix dry ingredients": "Mix dry ingredients",
        "Combine dry ingredients": "Mix dry ingredients",
        "Stir dry ingredients": "Mix dry ingredients",
        "Mix wet ingredients": "Mix wet ingredients",
        "Combine wet ingredients": "Mix wet ingredients",
        "Stir wet ingredients": "Mix wet ingredients",
        "Combine wet and dry ingredients": "Combine wet and dry ingredients",
        "Combine wet and dry ingredients until just mixed.": "Combine wet and dry ingredients",
        "Pour batter into a lined baking pan and bake for 20-25 minutes.": "Bake at 175°C for 20-25 minutes",
        "Preheat oven to 350°F (175°C).": "Preheat oven to 175°C",
        "Preheat oven to 350°F (176°C).": "Preheat oven to 175°C",
        "Preheat oven to 350Â°F (176Â°C).": "Preheat oven to 175°C",
        "Mix dry ingredients in one bowl and wet ingredients in another.": "Mix dry ingredients",
        "Combine wet and dry ingredients until just mixed.": "Combine wet and dry ingredients",
        "Pour batter into a lined baking pan and bake for 20-25 minutes.": "Bake at 175°C for 20-25 minutes",
        "Combine dry ingredients in a bowl.": "Mix dry ingredients",
        "Add wet ingredients (like almond milk and oil) and mix until combined.": "Mix wet ingredients",
        "Bake at 350°F (175°C) for 20-25 minutes.": "Bake at 175°C for 20-25 minutes",
        "Mix all ingredients until smooth.": "Combine wet and dry ingredients",
        "Pour into a lined pan and bake for 25-30 minutes.": "Bake at 175°C for 25-30 minutes",
        "Mix wet and dry ingredients separately, then combine.": "Combine wet and dry ingredients",
        "Bake for 25-30 minutes.": "Bake at 175°C for 25-30 minutes",
        "Combine wet ingredients, then mix in dry ingredients.": "Combine wet and dry ingredients",
        "Sift flour and cocoa powder.": "Mix dry ingredients",
        "Whip aquafaba and sugar until thick and glossy.": "Mix wet ingredients",
        "Melt chocolate and butter together.": "Mix wet ingredients",
        "Combine all mixtures, pour into pan, and bake for 34-37 minutes.": "Bake at 175°C for 34-37 minutes"
    }
    normalized = set(normalization_map.get(step, step) for step in steps)
    return list(normalized)

# Refine the objective function
def objective_function(X):
    # Decode the recipe
    recipe_ingredients, recipe_steps = decode_recipe(X)

    # Normalize steps to avoid duplication
    recipe_steps = normalize_steps(recipe_steps)

    # Ensure only one "Preheat oven" and one "Bake" step is selected
    preheat_steps = [step for step in recipe_steps if "Preheat oven" in step]
    bake_steps = [step for step in recipe_steps if "Bake" in step]
    if preheat_steps:
        recipe_steps = [step for step in recipe_steps if "Preheat oven" not in step]
        recipe_steps.append(preheat_steps[0])
    if bake_steps:
        recipe_steps = [step for step in recipe_steps if "Bake" not in step]
        recipe_steps.append(bake_steps[0])

    # Score based on unique ingredients used
    ingredient_score = sum(1 for ingredient in recipe_ingredients if ingredient in [ing['name'] for ing in ingredients])

    # Penalize for long lists of ingredients
    ingredient_penalty = len(recipe_ingredients)

    # Penalize for duplicate similar ingredients
    similar_ingredients = [
        ("butter", "almond butter"),
        ("sugar", "coconut sugar"),
        ("chocolate chips", "dark chocolate")
    ]
    duplicate_penalty = sum(1 for ing1, ing2 in similar_ingredients if ing1 in recipe_ingredients and ing2 in recipe_ingredients)

    # Score based on the presence of critical steps
    critical_step_score = sum(1 for step in critical_steps if step in recipe_steps)

    # Reward for logical order of steps (simple example)
    step_sequence_score = 0
    if 'Preheat oven' in recipe_steps:
        step_sequence_score += 1
    if 'Mix dry ingredients' in recipe_steps and 'Combine wet and dry ingredients' in recipe_steps:
        step_sequence_score += 1
    if 'Bake' in recipe_steps:
        step_sequence_score += 1

    # Combine scores
    total_score = ingredient_score + (critical_step_score * 2) + step_sequence_score - ingredient_penalty - duplicate_penalty
    return -total_score  # Minimization problem for GA

# Define the genetic algorithm model
model = ga(function=objective_function, dimension=num_ingredients + num_steps, variable_type='int', variable_boundaries=varbound)

# Run the genetic algorithm
model.run()

# Get the best solution
solution = model.output_dict['variable']

# Decode the best recipe
best_recipe_ingredients, best_recipe_steps = decode_recipe(solution)

# Normalize final steps to avoid duplication
best_recipe_steps = normalize_steps(best_recipe_steps)

# Ensure only one "Preheat oven" and one "Bake" step is selected
preheat_steps = [step for step in best_recipe_steps if "Preheat oven" in step]
bake_steps = [step for step in best_recipe_steps if "Bake" in step]
if preheat_steps:
    best_recipe_steps = [step for step in best_recipe_steps if "Preheat oven" not in step]
    best_recipe_steps.append(preheat_steps[0])
if bake_steps:
    best_recipe_steps = [step for step in best_recipe_steps if "Bake" not in step]
    best_recipe_steps.append(bake_steps[0])

# Print the best recipe
print("Ingredients:")
for ingredient, quantity in best_recipe_ingredients.items():
    print(f"{ingredient}: {quantity} grams")

print("\nSteps:")
for step in best_recipe_steps:
    print(step)

In [ ]:
# Define the file path
output_file_path = 'generated_recipe_'+str(counter)+'.txt'

# Write the ingredients and steps to the file
with open(output_file_path, 'w') as file:
    file.write("Ingredients:\n")
    for ingredient, quantity in best_recipe_ingredients.items():
        file.write(f"{ingredient}: {quantity} grams\n")
    
    file.write("\nSteps:\n")
    for step in best_recipe_steps:
        file.write(f"{step}\n")

print(f"Generated recipe has been written to {output_file_path}")
counter +=1